In [36]:
using StaticArrays
using LinearAlgebra
using Rotations
include("quaternions.jl")
using Quaternions
using BenchmarkTools
using Test

In [58]:
q1_ = normalize(@SVector rand(4))
q2_ = normalize(@SVector rand(4))
r = @SVector rand(3)

3-element SArray{Tuple{3},Float64,1,3} with indices SOneTo(3):
 0.12027330988983298
 0.19967162810951322
 0.3148016558090936 

# Quaternion Multiplication
## Rotations.jl

In [88]:
q1 = Quat(q1_...)
q2 = Quat(q2_...)
@show sizeof(q1)
@show sizeof(RotMatrix(q1))
q0 = q1*q2

@btime $q0 = $q1*$q2;

sizeof(q1) = 32
sizeof(RotMatrix(q1)) = 72
  9.929 ns (0 allocations: 0 bytes)


In [89]:
# Quaternions.jl
q1 = Quaternions.Quaternion(q1_...)
q2 = Quaternions.Quaternion(q2_...)
@show sizeof(q1)

q = q1*q2
@test q0.w ≈ q.s
@test q0.x ≈ q.v1
@test q0.y ≈ q.v2
@test q0.z ≈ q.v3

@btime $q = $q1*$q2;

sizeof(q1) = 40
  0.024 ns (0 allocations: 0 bytes)


In [90]:
# Our quaternions
q1 = Quaternion(q1_)
q2 = Quaternion(q2_)
@show sizeof(q1)

@inline function Base.:*(q2::Quaternion, q1::Quaternion)
    qmult(q2,q1)
end

function qmult(q2::Quaternion, q1::Quaternion)
    w1,v1 = scalar(q1), vector(q1)
    w2,v2 = scalar(q2),vector(q2)
    w = w1*w2 - v1'v2
    v = w1*v2 + w2*v1 + cross(v2, v1)
    return Quaternion(w,v)
end

q = q1*q2
# @test q0.w ≈ q.w
# @test q0.x ≈ q.v1
# @test q0.y ≈ q.v2
# @test q0.z ≈ q.v3


@btime $q = qmult($q2,$q1);
@btime $q = $q2*$q1;

sizeof(q1) = 32
  12.310 ns (0 allocations: 0 bytes)
  12.293 ns (0 allocations: 0 bytes)


# Vector Rotation
## Rotations.jl

In [91]:
q1 = Quat(q1_...)
r0 = q1*r
@btime $r0 = $q1*$r;

  0.024 ns (0 allocations: 0 bytes)


# Quaternions.jl

In [92]:
q1 = Quaternions.Quaternion(q1_...)
rhat = Quaternions.Quaternion(Vector(r))
r1 = q1*rhat*conj(q1)
@test [r1.v1, r1.v2, r1.v3] ≈ r0
@btime $r1 = $q1*$rhat*conj($q1)

  0.024 ns (0 allocations: 0 bytes)


Quaternions.Quaternion{Float64}(-6.938893903907228e-18, 0.2087775247318696, 0.30261193054021607, 0.1351757201033663, false)

## Our Quaternions

In [93]:
q1 = Quaternion(q1_)
@test rotate(r,q1) ≈ r0
@btime $r2 = rotate($r,$q1);

  5.635 ns (0 allocations: 0 bytes)
